In [1]:
!gdown 1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3

Downloading...
From: https://drive.google.com/uc?id=1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3
To: c:\Users\Lan Vi\Downloads\AIO_Exercises\Module3\week4\Housing.csv

  0%|          | 0.00/30.0k [00:00<?, ?B/s]
100%|██████████| 30.0k/30.0k [00:00<00:00, 700kB/s]


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
data_path = "/content/Housing.csv"
df = pd.read_csv(data_path)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
df.isna().sum()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [6]:
category_cols = df.select_dtypes(include="object").columns.to_list()
category_cols

['mainroad',
 'guestroom',
 'basement',
 'hotwaterheating',
 'airconditioning',
 'prefarea',
 'furnishingstatus']

In [9]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

ordinal_encoder = OrdinalEncoder()
df[category_cols] = ordinal_encoder.fit_transform(df[category_cols])
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1.0,0.0,0.0,0.0,1.0,2,1.0,0.0
1,12250000,8960,4,4,4,1.0,0.0,0.0,0.0,1.0,3,0.0,0.0
2,12250000,9960,3,2,2,1.0,0.0,1.0,0.0,0.0,2,1.0,1.0
3,12215000,7500,4,2,2,1.0,0.0,1.0,0.0,1.0,3,1.0,0.0
4,11410000,7420,4,1,2,1.0,1.0,1.0,0.0,1.0,2,0.0,0.0


In [10]:
normalizer = StandardScaler()
normed_data = normalizer.fit_transform(df)
normed_data = pd.DataFrame(normed_data, columns=df.columns)
normed_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,0.405623,-0.465315,-0.734539,-0.219265,1.472618,1.517692,1.804941,-1.406286
1,4.004484,1.757010,1.403419,5.405809,2.532024,0.405623,-0.465315,-0.734539,-0.219265,1.472618,2.679409,-0.554035,-1.406286
2,4.004484,2.218232,0.047278,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,-0.679063,1.517692,1.804941,-0.091662
3,3.985755,1.083624,1.403419,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,1.472618,2.679409,1.804941,-1.406286
4,3.554979,1.046726,1.403419,-0.570187,0.224410,0.405623,2.149083,1.361397,-0.219265,1.472618,1.517692,-0.554035,-1.406286


In [11]:
X, y = normed_data.drop("price", axis=1), normed_data["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=True
)

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
depths = range(1, 10)
min_mse = None
best_depth = None
for depth in depths:
  tree = DecisionTreeRegressor(max_depth=depth)
  tree.fit(X_train, y_train)
  y_pred = tree.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Depth: {depth}, MSE: {mse}")
  if min_mse is None or mse < min_mse:
    min_mse = mse
    best_depth = depth

print(f"Best depth: {best_depth}, MSE: {min_mse}")

Depth: 1, MSE: 0.8104164672132039
Depth: 2, MSE: 0.6624731445240474
Depth: 3, MSE: 0.6226684506223363
Depth: 4, MSE: 0.5861341802065629
Depth: 5, MSE: 0.5556559601395509
Depth: 6, MSE: 0.5662896289712717
Depth: 7, MSE: 0.6208182349117655
Depth: 8, MSE: 0.6497536058879737
Depth: 9, MSE: 0.6605447198864042
Best depth: 5, MSE: 0.5556559601395509


In [26]:
random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MSE: 0.4013639246423486
MAE: 0.4624147171532586


In [22]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

# AdaBoost
ada_boost = AdaBoostRegressor()
ada_boost.fit(X_train, y_train)
y_pred = ada_boost.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"AdaBoost MSE: {mse}")

# Gradient Boosting
gradient_boost = GradientBoostingRegressor()
gradient_boost.fit(X_train, y_train)
y_pred = gradient_boost.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Gradient Boosting MSE: {mse}")

AdaBoost MSE: 0.5831238966120467
Gradient Boosting MSE: 0.4000386840801441
